## Reference: 
1. Selenium and BS4: https://www.edureka.co/blog/web-scraping-with-python/
2. Potential automatically more button clicking: https://www.pythonfixing.com/2022/03/fixed-scraping-webpage-with-show-more.html


In [49]:
import scrapy
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import json 


In [2]:
# driver = webdriver.Chrome("/opt/homebrew/Caskroom/chromedriver/103.0.5060.134/chromedriver")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
## bestbuy latops home page 
driver.get("https://www.bestbuy.ca/en-ca/category/laptops/36711")

content = driver.page_source
soup = BeautifulSoup(content)


In [4]:
## Parse all the products on the main page, manually clicked the show more button several times to get 10 pages 
domin_name = 'https://www.bestbuy.ca'

list_products_on_one_page = []

container = soup.find('div', attrs={'class':'productsRow_DcaXn row_1mOdd'})
for prod in container.findAll('a', href=True,attrs={'class':'link_3hcyN'}): 
    list_products_on_one_page.append(domin_name + prod['href'])

## Filter out the beginning 5 general pages(desktops, 2-in-1s, ...)
list_products_on_one_page_short = list_products_on_one_page[5:]


In [68]:
# Parse the products names and specs as needed

output = []

for i, link in enumerate(list_products_on_one_page_short):
    driver.get(str(link))
    content = driver.page_source
    soup = BeautifulSoup(content)

    info = ''
    try: 
        prod_name = soup.find('h1', attrs={'class':'productName_2KoPa'},string=True).text
        info += prod_name + ' '
        for i in soup.find_all('div', attrs={'class':'itemContainer_uqm6b'}):
            item_name = i.find('div', attrs={'class':'itemName_GaNqp'}).text
            item_value = i.find('div', attrs={'class':'itemValue_3FLTX'}).text
            # specs_table[item_name] = item_value
            info += item_name + ': ' + item_value + ', '
            # products.append(name.text)

        output.append(info)
    except:
        print(f"This page {i} can't be parsed using this approach")

This page can't be parsed using this approach
This page can't be parsed using this approach
This page can't be parsed using this approach
This page can't be parsed using this approach
This page can't be parsed using this approach


In [71]:
## Write to txt file for azure language service
for idx, record in enumerate(output):

    try:
        with open(f'raw_txt/product_{idx}.txt', 'w') as f:
            f.write(record)
    except FileNotFoundError:
        print("The directory does not exist")

In [ ]:
## potential way to use multiple windows, not implemented till the end 

list_of_links = driver.find_element(By.CLASS_NAME, "link_3hcyN")
cur_win = driver.current_window_handle # get current/main window

for link in list_of_links:
    (domin_name + link['href']).click()
    driver.switch_to_window([win for win in driver.window_handles if win !=cur_win][0]) # switch to new window
    date = driver.find_elements_by_class_name("productName_2KoPa")
    # for d in date:
    print(date.text)
    driver.close() # close new window
    driver.switch_to_window(cur_win) # switch back to main window
